In [1]:
#!/usr/bin/env sage
"""
Part 1: Polynomial Benchmark Testing Module
Run this once to collect all benchmark data, then save results to file
"""

from sage.all import *
import itertools
import subprocess
import tempfile
import os
import json
import time
import signal
from datetime import datetime
import numpy as np
import pickle

# Matrix multiplication exponent
OMEGA = 2  # Using best known bound

# Field definitions
STANDARD_PRIME = 2147483489  # Supports msolve
LARGE_PRIME = 9223372036854775783  # ≈2^63, no msolve support

def fuss_catalan(n, d):
    """Compute the Fuss-Catalan number D = (1/(n(d-1)+1)) * C(nd, n)"""
    if n == 0 or d == 0:
        return 1
    return binomial(n*d, n) / (n*(d-1) + 1)

def compute_f4_fglm_complexity(n, d, omega=OMEGA):
    """Compute theoretical F4+FGLM complexity: O(n * d^(n*omega))"""
    d_reg = n*(d-1) + 1
    return float(binomial(n+d_reg, n)**omega + n * d**(n * omega))


In [2]:
log(compute_f4_fglm_complexity(50,3),2).n()
fuss_catalan(4, 3)

55

In [3]:
def count_paths_linear_boundary(c, d, mu, nu):
    """
    计算单一线性边界 x ≤ mu*y + nu 下的格路径数
    使用ballot定理的推广形式
    
    参数:
    - c, d: 终点坐标 (c,d)
    - mu, nu: 边界线 x = mu*y + nu 的参数
    
    返回: 路径数
    """
    if d < 0 or c < 0:
        return 0
    
    # 检查终点是否满足边界条件
    if c > mu * d + nu:
        return 0
    
    # 使用ballot定理推广公式
    # 路径数 = (c - mu*d - nu + 1)/(c + d - mu*d - nu + 1) * C(c + d - mu*d - nu + 1, d)
    
    denominator = c + d - mu * d - nu + 1
    if denominator <= 0:
        return 0
    
    numerator = c - mu * d - nu + 1
    if numerator <= 0:
        return 0
    
    from math import comb
    binomial_coeff = comb(denominator, d)
    
    return (numerator * binomial_coeff) // denominator

def count_catalan_paths(n):
    """
    正确计算卡塔兰数：从(0,0)到(n,n)且不超过对角线y=x的路径数
    使用 y ≤ x 约束，这与定理10.6.1的边界方向不同
    """
    if n <= 0:
        return 1
    
    # DP: dp[i][j] = 到达(i,j)且满足路径上所有点都有y≤x的路径数
    dp = {}
    dp[(0, 0)] = 1
    
    for i in range(n + 1):
        for j in range(min(i + 1, n + 1)):  # j ≤ i 确保 y ≤ x
            if (i, j) not in dp:
                dp[(i, j)] = 0
            
            if dp[(i, j)] > 0:
                # 向右移动到 (i+1, j)
                if i + 1 <= n and j <= i + 1:  # 确保新点也满足 y ≤ x
                    if (i + 1, j) not in dp:
                        dp[(i + 1, j)] = 0
                    dp[(i + 1, j)] += dp[(i, j)]
                
                # 向上移动到 (i, j+1)  
                if j + 1 <= n and j + 1 <= i:  # 确保新点满足 y ≤ x
                    if (i, j + 1) not in dp:
                        dp[(i, j + 1)] = 0
                    dp[(i, j + 1)] += dp[(i, j)]
    
    return dp.get((n, n), 0)

def count_paths_piecewise_boundary_simple(c, d, boundaries):
    """
    修正版分段边界路径计数，改进边界约束检查逻辑
    
    参数:
    - c, d: 终点坐标
    - boundaries: 边界定义列表，每个元素为 (y_start, y_end, mu, nu)
                 表示在 y ∈ [y_start, y_end] 区间内边界为 x ≤ mu*y + nu
    
    返回: 路径数
    """
    if len(boundaries) == 1:
        # 只有一段边界
        y_start, y_end, mu, nu = boundaries[0]
        if y_start == 0 and y_end == d:
            return count_paths_linear_boundary(c, d, mu, nu)
    
    # 多段边界：使用动态规划
    dp = {}
    dp[(0, 0)] = 1  # 起点
    
    for y in range(d + 1):
        # 找当前y坐标对应的边界
        current_boundary = None
        for y_start, y_end, mu, nu in boundaries:
            if y_start <= y <= y_end:
                current_boundary = (mu, nu)
                break
        
        if current_boundary is None:
            continue
            
        mu, nu = current_boundary
        max_x_allowed = mu * y + nu
        
        # 处理当前y层的所有可能x坐标
        for x in range(max_x_allowed + 1):
            if dp.get((x, y), 0) > 0:
                # 向右移动
                if x + 1 <= c:
                    # 检查 (x+1, y) 是否满足边界约束
                    if x + 1 <= max_x_allowed:
                        dp[(x + 1, y)] = dp.get((x + 1, y), 0) + dp[(x, y)]
                
                # 向上移动
                if y + 1 <= d:
                    # 找下一层y+1的边界约束
                    next_boundary = None
                    for y_start, y_end, mu_next, nu_next in boundaries:
                        if y_start <= y + 1 <= y_end:
                            next_boundary = (mu_next, nu_next)
                            break
                    
                    if next_boundary:
                        mu_next, nu_next = next_boundary
                        max_x_next = mu_next * (y + 1) + nu_next
                        if x <= max_x_next:
                            dp[(x, y + 1)] = dp.get((x, y + 1), 0) + dp[(x, y)]
    
    return dp.get((c, d), 0)

def example_piecewise_boundary():
    """
    使用定理10.6.1的递归公式实现（简化版本）
    """
    print("=== 分段线性边界格路径计数（简化实现）===\n")
    
    # 测试1: 两段边界
    print("测试1: 两段边界")
    print("第1段: x = y, 0 ≤ y ≤ 2")
    print("第2段: x = 2y, 2 < y ≤ 4")
    
    boundaries1 = [
        (0, 2, 1, 0),  # x = y, 0 ≤ y ≤ 2
        (3, 4, 2, 0)   # x = 2y, 3 ≤ y ≤ 4
    ]
    
    result1 = count_paths_piecewise_boundary_simple(6, 4, boundaries1)
    print(f"从(0,0)到(6,4)的路径数: {result1}\n")
    
    # 测试2: 单段边界（验证）
    print("测试2: 单段边界验证")
    boundaries2 = [(0, 3, 2, 1)]  # x = 2y + 1, 0 ≤ y ≤ 3
    
    result2a = count_paths_piecewise_boundary_simple(5, 3, boundaries2)
    result2b = count_paths_linear_boundary(5, 3, 2, 1)
    
    print(f"DP方法结果: {result2a}")
    print(f"直接公式结果: {result2b}")
    print(f"结果一致: {result2a == result2b}\n")
    
    # 测试3: 修正的边界测试
    print("测试3: 边界测试说明")
    print("注意：经典卡塔兰数要求 y ≤ x，但定理10.6.1处理 x ≤ μy+ν 形式的边界")
    print("这里测试一些定理适用的边界情况：")
    
    # 测试 x ≤ 2y 的情况（更宽松的边界）
    for n in range(1, 5):
        boundaries_wide = [(0, n, 2, 0)]  # x ≤ 2y
        paths_wide = count_paths_piecewise_boundary_simple(n, n, boundaries_wide)
        
        # 理论上应该等于不受约束的路径数 C(2n,n)
        from math import comb
        unrestricted = comb(2*n, n)
        
        print(f"n={n}: x≤2y边界路径数={paths_wide}, 无约束路径数={unrestricted}")
    
    print("对比：正确的卡塔兰数计算（y ≤ x 约束）")
    for n in range(1, 7):
        catalan_correct = count_catalan_paths(n)
        catalan_formula = catalan_number(n)
        print(f"n={n}: DP计算={catalan_correct}, 公式计算={catalan_formula}, 匹配: {catalan_correct == catalan_formula}")
    
    print()
    
    # 测试 x ≤ y + k 的情况
    print("测试 x ≤ y + k 边界:")
    for k in [0, 1, 2]:
        boundaries_shifted = [(0, 3, 1, k)]  # x ≤ y + k
        paths_shifted = count_paths_piecewise_boundary_simple(3, 3, boundaries_shifted)
        print(f"k={k}: 从(0,0)到(3,3)在x≤y+{k}边界下的路径数={paths_shifted}")
    
    print()
    
    # 测试4: 复杂三段边界
    print("测试4: 三段边界")
    print("第1段: x = 3y, 0 ≤ y ≤ 1")
    print("第2段: x = y + 2, 1 < y ≤ 3") 
    print("第3段: x = 2y, 3 < y ≤ 5")
    
    boundaries3 = [
        (0, 1, 3, 0),   # x = 3y, 0 ≤ y ≤ 1
        (2, 3, 1, 2),   # x = y + 2, 2 ≤ y ≤ 3
        (4, 5, 2, 0)    # x = 2y, 4 ≤ y ≤ 5
    ]
    
    result3 = count_paths_piecewise_boundary_simple(8, 5, boundaries3)
    print(f"从(0,0)到(8,5)的路径数: {result3}\n")

def catalan_number(n):
    """计算第n个卡塔兰数"""
    if n <= 0:
        return 1
    from math import comb
    return comb(2*n, n) // (n + 1)

def visualize_boundary(boundaries, c, d):
    """
    可视化边界和一些示例路径
    """
    try:
        import matplotlib.pyplot as plt
        
        plt.figure(figsize=(10, 8))
        
        # 绘制边界
        colors = ['red', 'blue', 'green', 'orange', 'purple']
        for i, (y_start, y_end, mu, nu) in enumerate(boundaries):
            y_range = list(range(y_start, y_end + 1))
            x_range = [min(mu * y + nu, c + 2) for y in y_range]
            
            plt.plot(x_range, y_range, color=colors[i % len(colors)], 
                    linewidth=3, label=f'段{i+1}: x = {mu}y + {nu}')
        
        # 绘制格子
        for i in range(c + 3):
            plt.axvline(x=i, color='lightgray', linestyle='-', alpha=0.3)
        for j in range(d + 2):
            plt.axhline(y=j, color='lightgray', linestyle='-', alpha=0.3)
        
        # 标记起点和终点
        plt.plot(0, 0, 'go', markersize=10, label='起点(0,0)')
        plt.plot(c, d, 'ro', markersize=10, label=f'终点({c},{d})')
        
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title(f'分段线性边界：从(0,0)到({c},{d})')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.axis('equal')
        plt.xlim(-0.5, c + 1.5)
        plt.ylim(-0.5, d + 1.5)
        plt.show()
        
    except ImportError:
        print("需要matplotlib来绘图")

def solve_inequality_system(t_values):
    """
    求解不等式组的非负整数解个数
    
    不等式组（修改为≤）：
    x₀ ≤ t₀
    x₀ + x₁ ≤ t₁  
    x₀ + x₁ + x₂ ≤ t₂
    ...
    x₀ + x₁ + ... + xₙ ≤ tₙ
    
    参数:
    - t_values: 列表 [t₀, t₁, t₂, ..., tₙ]
    
    返回: 解的个数
    """
    if not t_values or any(t < 0 for t in t_values):
        return 0
    
    n = len(t_values) - 1  # 变量个数是n+1个：x₀, x₁, ..., xₙ
    
    # 转化为格路径问题：
    # 状态：dp[i][s] = 前i+1个累积和中，第i个累积和为s的方案数
    # 约束：0 ≤ S₀ ≤ S₁ ≤ ... ≤ Sₙ，且 S_i ≤ t_i
    
    # DP数组：dp[i][s] 表示到第i个位置，累积和为s的方案数
    dp = {}
    
    # 初始化：第0个累积和S₀ = x₀，范围是0到t₀
    for s in range(t_values[0] + 1):
        dp[(0, s)] = 1
    
    # 动态规划转移
    for i in range(1, n + 1):  # i从1到n
        t_i = t_values[i]  # 当前约束 S_i ≤ t_i
        
        for s in range(t_i + 1):  # 当前累积和s的范围：0到t_i
            dp[(i, s)] = 0
            
            # 枚举上一个累积和的可能值
            for prev_s in range(min(s + 1, t_values[i-1] + 1)):  # prev_s ≤ s 且 prev_s ≤ t_{i-1}
                if (i-1, prev_s) in dp:
                    dp[(i, s)] += dp[(i-1, prev_s)]
    
    # 统计所有可能的最终累积和
    total = 0
    for s in range(t_values[n] + 1):
        total += dp.get((n, s), 0)
    
    return total

def solve_inequality_system_optimized(t_values):
    """
    优化版本：减少内存使用，只保留当前和前一层的DP状态
    """
    if not t_values or any(t < 0 for t in t_values):
        return 0
    
    n = len(t_values) - 1
    
    # 只保留当前层的DP状态
    prev_dp = {}
    
    # 初始化第0层
    for s in range(t_values[0] + 1):
        prev_dp[s] = 1
    
    # 逐层计算
    for i in range(1, n + 1):
        curr_dp = {}
        t_i = t_values[i]
        
        for s in range(t_i + 1):
            curr_dp[s] = 0
            # 累加所有合法的前一状态
            for prev_s in range(min(s + 1, t_values[i-1] + 1)):
                if prev_s in prev_dp:
                    curr_dp[s] += prev_dp[prev_s]
        
        prev_dp = curr_dp
    
    return sum(prev_dp.values())

def analyze_solution_structure(t_values, show_details=False):
    """
    分析解的结构，可选择显示具体解
    修改为≤约束
    """
    if not t_values or len(t_values) > 4:  # 限制输出长度
        print("输入无效或过长，无法详细分析")
        return
    
    print(f"\n=== 分析t={t_values}的解结构（≤约束）===")
    
    n = len(t_values) - 1
    solutions = []
    
    def backtrack(curr_solution, curr_sum, pos):
        """回溯生成所有解"""
        if pos == n + 1:
            solutions.append(curr_solution[:])
            return
        
        # 对于第pos个变量x_pos，枚举所有可能值
        # 约束：curr_sum + x_pos ≤ t_values[pos]
        max_val = t_values[pos] - curr_sum
        
        for val in range(max(0, max_val + 1)):
            if curr_sum + val <= t_values[pos]:
                curr_solution.append(val)
                backtrack(curr_solution, curr_sum + val, pos + 1)
                curr_solution.pop()
    
    backtrack([], 0, 0)
    
    print(f"总解数: {len(solutions)}")
    
    if show_details and len(solutions) <= 20:
        print("所有解:")
        for i, sol in enumerate(solutions):
            sums = [sum(sol[:j+1]) for j in range(len(sol))]
            constraints_check = [sums[j] <= t_values[j] for j in range(len(sums))]
            print(f"  解{i+1}: {sol} → 累积和: {sums} → 约束满足: {constraints_check}")
    elif len(solutions) > 20:
        print("解太多，不显示详细信息")
    
    return len(solutions)

def dixon_size(a_values, show_details=False):
    """
    处理输入序列：排序并生成累积和，然后计算解的个数
    
    参数:
    - a_values: 输入序列 [a₀, a₁, a₂, ..., aₙ]
    - show_details: 是否显示详细过程
    
    处理步骤:
    1. 将a_values从大到小排列得到 [b₀, b₁, ..., bₙ]
    2. 设置 t₀=b₀, t₁=b₀+b₁, t₂=b₀+b₁+b₂, ...
    3. 求解不等式组解的个数
    
    返回: 解的个数
    """
    if not a_values:
        return 0
    
    # 步骤1：从大到小排序，去掉最小值
    b_values = sorted(a_values, reverse=True)[:-1]
    
    # 步骤2：生成累积和序列
    t_values = []
    cumsum = 0
    for b in b_values:
        b -= 1
        cumsum += b
        t_values.append(cumsum)
    
    if show_details:
        print(f"输入序列: {a_values}")
        print(f"排序后序列: {b_values}")
        print(f"累积和t_values: {t_values}")
        print()
    
    # 步骤3：求解不等式组
    result = solve_inequality_system_optimized(t_values)
    
    if show_details:
        print(f"不等式组：")
        for i in range(len(t_values)):
            vars_str = " + ".join([f"x_{j}" for j in range(i + 1)])
            print(f"  {vars_str} ≤ {t_values[i]}")
        print(f"\n解的个数: {result}")
        
        # 如果序列不太大，显示详细解结构
        if len(t_values) <= 4:
            analyze_solution_structure(t_values, show_details=True)
    
    return result

def test_updated_functions():
    """
    测试更新后的函数
    """
    print("=== 测试更新后的不等式组求解（≤约束）===\n")
    
    # 测试1：基本功能验证
    print("测试1: t = [1, 2, 3]")
    print("不等式组：x₀ ≤ 1, x₀+x₁ ≤ 2, x₀+x₁+x₂ ≤ 3")
    t1 = [1, 2, 3]
    result1 = solve_inequality_system_optimized(t1)
    print(f"解的个数: {result1}")
    analyze_solution_structure(t1, show_details=True)
    
    # 测试2：序列处理函数
    print("\n" + "="*50)
    print("测试2: 序列处理函数")
    
    # 示例1：简单序列
    a1 = [3, 1, 2]
    result2 = dixon_size(a1, show_details=True)
    
    print("\n" + "-"*30)
    
    # 示例2：更复杂序列
    a2 = [1, 4, 2, 5, 3]
    result3 = dixon_size(a2, show_details=True)
    
    print("\n" + "-"*30)
    
    # 示例3：相同数字的序列
    a3 = [2, 2, 2, 2]
    result4 = dixon_size(a3, show_details=True)
    
    print("\n" + "="*50)
    print("批量测试不同序列长度:")
    
    import random
    random.seed(None)  # 固定随机种子以便重现结果
    
    for length in range(2, 8):
        # 生成随机序列
        a_random = [random.randint(1, 10) for _ in range(length)]
        count = dixon_size(a_random, show_details=False)
        print(f"长度{length}, 序列{a_random}: 解数 = {count}")

def compare_with_original_constraints():
    """
    对比原始约束(<)和修改后约束(≤)的结果差异
    """
    print("\n=== 对比 < 约束 vs ≤ 约束 ===")
    
    test_cases = [
        [1, 2, 3],
        [2, 4, 6],
        [1, 3, 6, 10],
        [5, 5, 5, 5]
    ]
    
    def solve_strict_inequality(t_values):
        """原始的<约束求解"""
        return solve_inequality_system_optimized([t - 1 for t in t_values])
    
    for t_values in test_cases:
        result_leq = solve_inequality_system_optimized(t_values)  # ≤ 约束
        result_lt = solve_strict_inequality(t_values)  # < 约束
        
        print(f"t = {t_values}:") 
        print(f"  ≤ 约束解数: {result_leq}")
        print(f"  < 约束解数: {result_lt}")
        print(f"  差异: {result_leq - result_lt}")
        print()

def dixon_complexity(a_values, n, omega):
    size = dixon_size(a1, show_details=False)
    d = 0
    m = len(a_values)
    if (m == n+1):
        d = 1
    elif (m == n):
        for a in a_values:
            d += a
    elif (m < n):
        for a in a_values:
            d += a
        d = (d+1)^(n-m+1)
    else:
        return 0
    return log(d*(size)^omega,2)

a1 = [2, 3, 4, 5]
omega = 2.3
for n in range(10):
    print(dixon_complexity(a1, n, omega))
# 运行测试
#test_updated_functions()
#compare_with_original_constraints()

0
0
0
17.0415991531167
20.8489540751743
24.8553803443337
28.7622709399423
32.6691615355508
36.5760521311593
40.4829427267678


In [12]:
a1 = [49, 49, 49]
result2 = dixon_size(a1, show_details=False)
result2

3577

In [31]:
def dixon_complexity(a_values, n, omega):
    size = dixon_size(a1, show_details=False)
    d = 0
    m = len(a_values)
    if (m == n+1):
        d = 1
    elif (m == n):
        for a in a_values:
            d += a
    elif (m < n):
        for a in a_values:
            d += a
        d = (d+1)^(n-m+1)
    else:
        return 0
    return log(d*(size)^omega,2)
    

In [35]:
a1 = [49, 40, 49]
n = 4
omega = 2.3
for n in range(10):
    print(dixon_complexity(a1, n, omega))

0
0
27.1504291268890
34.2589535836672
41.3883112723360
48.5072523450595
55.6261934177830
62.7451344905066
69.8640755632301
76.9830166359536
